In [ ]:
# 3.5 PyTest — маркировка
# 3_5_2 Маркировка тестов часть 1
# Например, мы можем выбрать небольшое количество критичных тестов (smoke), которые нужно запускать на каждый коммит разработчиков,
# а остальные тесты обозначить как регрессионные (regression) и запускать их только перед релизом.
# Для маркировки теста нужно написать декоратор вида @pytest.mark.mark_name, где mark_name — произвольная строка.

# Давайте разделим тесты в одном из предыдущих примеров на smoke и regression.
test_fixture8_352.py:

In [1]:
# Чтобы запустить тест с нужной маркировкой, нужно передать в командной строке параметр -m и нужную метку:
# pytest -s -v -m smoke test_fixture8_352.py
# А что означает -s -v -m ?
# флаги для запуска: -s выводить в консоль принты, -v выводить полный отчёт,  -m запускать маркированные тесты.

import pytest
from selenium import webdriver

link = "http://selenium1py.pythonanywhere.com/"


@pytest.fixture(scope="function")
def browser():
    print("\nstart browser for test..")
    browser = webdriver.Chrome()
    yield browser
    print("\nquit browser..")
    browser.quit()


class TestMainPage1():

    @pytest.mark.smoke
    def test_guest_should_see_login_link(self, browser):
        browser.get(link)
        browser.find_element_by_css_selector("#login_link")

    @pytest.mark.regression
    def test_guest_should_see_basket_link_on_the_main_page(self, browser):
        browser.get(link)
        browser.find_element_by_css_selector(".basket-mini .btn-group > a")

In [ ]:
# При этом вы увидите warning, то есть предупреждение:

PytestUnknownMarkWarning: Unknown pytest.mark.smoke - is this a typo?
    You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/latest/mark.html
    PytestUnknownMarkWarning
# PytestUnknownMarkWarning: Unknown pytest.mark.smoke — это опечатка?
#      Вы можете зарегистрировать пользовательские метки, чтобы избежать этого предупреждения — подробности см. на странице https://docs.pytest.org/en/latest/mark.html.
#      PytestUnknownMarkWarning
# Это предупреждение появилось потому, что в последних версиях PyTest настоятельно рекомендуется регистрировать
# метки явно перед использованием.

# наконец-то получилось выводить тесты в удобочитаемом виде без всяких warnings и тд:
# pytest -s -v -m smoke test_fixture8.py -q --tb=no -p no:warnings

In [ ]:
# 3_5_3 Маркировка тестов часть 2
# Инверсия
# Чтобы запустить все тесты, не имеющие заданную маркировку, можно использовать инверсию.
# Для запуска всех тестов, не отмеченных как smoke, нужно выполнить команду:
pytest -s -v -m "not smoke" test_fixture8.py

# Объединение тестов с разными маркировками
# Для запуска тестов с разными метками можно использовать логическое ИЛИ. Запустим smoke и regression-тесты:
pytest -s -v -m "smoke or regression" test_fixture8.py

In [ ]:
# Выбор тестов, имеющих несколько маркировок
# Предположим, у нас есть smoke-тесты, которые нужно запускать только для определенной операционной системы,
# например, для Windows 10. Зарегистрируем метку win10 в файле pytest.ini, а также добавим к одному из тестов эту метку.

# pytest.ini:

# [pytest]
# markers =
#     smoke: marker for smoke tests
#     regression: marker for regression tests
#     win10
# test_fixture81_353.py:

In [ ]:
import pytest
from selenium import webdriver

link = "http://selenium1py.pythonanywhere.com/"


@pytest.fixture(scope="function")
def browser():
    print("\nstart browser for test..")
    browser = webdriver.Chrome()
    yield browser
    print("\nquit browser..")
    browser.quit()


class TestMainPage1:

    @pytest.mark.smoke
    def test_guest_should_see_login_link(self, browser):
        browser.get(link)
        browser.find_element_by_css_selector("#login_link")

    @pytest.mark.smoke
    @pytest.mark.win10
    def test_guest_should_see_basket_link_on_the_main_page(self, browser):
        browser.get(link)
        browser.find_element_by_css_selector(".basket-mini .btn-group > a")

# Чтобы запустить только smoke-тесты для Windows 10, нужно использовать логическое И:

# pytest -s -v -m "smoke and win10" test_fixture81_353.py
# Должен выполнится тест test_guest_should_see_basket_link_on_the_main_page. 

In [ ]:
# 3_5_4 Пропуск тестов
# В PyTest есть стандартные метки, которые позволяют пропустить тест при сборе тестов для запуска
# (то есть не запускать тест) или запустить, но отметить особенным статусом тот тест,
# который ожидаемо упадёт из-за наличия бага, чтобы он не влиял на результаты прогона всех тестов.
# Эти метки не требуют дополнительного объявления в pytest.ini.

# Пропустить тест
# Итак, чтобы пропустить тест, его отмечают в коде как @pytest.mark.skip:

In [ ]:
import pytest
from selenium import webdriver

link = "http://selenium1py.pythonanywhere.com/"

@pytest.fixture(scope="function")
def browser():
    print("\nstart browser for test..")
    browser = webdriver.Chrome()
    yield browser
    print("\nquit browser..")
    browser.quit()


class TestMainPage1():

    @pytest.mark.skip
    def test_guest_should_see_login_link(self, browser):
        browser.get(link)
        browser.find_element_by_css_selector("#login_link")

    def test_guest_should_see_basket_link_on_the_main_page(self, browser):
        browser.get(link)
        browser.find_element_by_css_selector(".basket-mini .btn-group > a")
        
# В результатах теста мы увидим, что один тест был пропущен, а другой успешно прошёл: "1 passed, 1 skipped".

In [ ]:
# 3_5_5 XFail: помечать тест как ожидаемо падающий
# Отметить тест как падающий

# Теперь добавим в наш тестовый класс тест, который проверяет наличие кнопки "Избранное":

def test_guest_should_see_search_button_on_the_main_page(self, browser): 
     browser.get(link)
     browser.find_element_by_css_selector("button.favorite")
# Предположим, что такая кнопка должна быть, но из-за изменений в коде она пропала.
# Пока разработчики исправляют баг, мы хотим, чтобы результат прогона ﻿всех ﻿наших тестов был успешен,
# но падающий тест помечался соответствующим образом, чтобы про него не забыть.
# Добавим маркировку @pytest.mark.xfail для падающего теста.

In [ ]:
# test_fixture10_355.py

import pytest
from selenium import webdriver

link = "http://selenium1py.pythonanywhere.com/"


@pytest.fixture(scope="function")
def browser():
    print("\nstart browser for test..")
    browser = webdriver.Chrome()
    yield browser
    print("\nquit browser..")
    browser.quit()


class TestMainPage1():

    def test_guest_should_see_login_link(self, browser):
        browser.get(link)
        browser.find_element_by_css_selector("#login_link")

    def test_guest_should_see_basket_link_on_the_main_page(self, browser):
        browser.get(link)
        browser.find_element_by_css_selector(".basket-mini .btn-group > a")

    @pytest.mark.xfail
    def test_guest_should_see_search_button_on_the_main_page(self, browser):
        browser.get(link)
        browser.find_element_by_css_selector("button.favorite")

# Запустим наши тесты:
# pytest -v test_fixture10_355.py
# Наш упавший тест теперь отмечен как xfail, но результат прогона тестов помечен как успешный

In [ ]:
# Когда баг починят, мы это узнаем, ﻿﻿так как теперь тест будет отмечен как XPASS
# (“unexpectedly passing” — неожиданно проходит).
# После этого маркировку xfail для теста можно удалить. Кстати, к маркировке xfail можно добавлять параметр reason.
# Чтобы увидеть это сообщение в консоли, при запуске нужно добавлять параметр pytest -rx.

# test_fixture10a_355.py

import pytest
from selenium import webdriver

link = "http://selenium1py.pythonanywhere.com/"


@pytest.fixture(scope="function")
def browser():
    print("\nstart browser for test..")
    browser = webdriver.Chrome()
    yield browser
    print("\nquit browser..")
    browser.quit()


class TestMainPage1():

    def test_guest_should_see_login_link(self, browser):
        browser.get(link)
        browser.find_element_by_css_selector("#login_link")

    def test_guest_should_see_basket_link_on_the_main_page(self, browser):
        browser.get(link)
        browser.find_element_by_css_selector(".basket-mini .btn-group > a")

    @pytest.mark.xfail(reason="fixing this bug right now")
    def test_guest_should_see_search_button_on_the_main_page(self, browser):
        browser.get(link)
        browser.find_element_by_css_selector("button.favorite")

# Запустим наши тесты:
# pytest -rx -v test_fixture10a_355.py
# Сравните вывод в первом и во втором случае.

In [ ]:
# XPASS-тесты
# Поменяем селектор в последнем тесте, чтобы тест начал проходить.
# test_fixture10b_355.py:

import pytest
from selenium import webdriver

link = "http://selenium1py.pythonanywhere.com/"


@pytest.fixture(scope="function")
def browser():
    print("\nstart browser for test..")
    browser = webdriver.Chrome()
    yield browser
    print("\nquit browser..")
    browser.quit()


class TestMainPage1():

    def test_guest_should_see_login_link(self, browser):
        browser.get(link)
        browser.find_element_by_css_selector("#login_link")

    def test_guest_should_see_basket_link_on_the_main_page(self, browser):
        browser.get(link)
        browser.find_element_by_css_selector(".basket-mini .btn-group > a")

    @pytest.mark.xfail(reason="fixing this bug right now")
    def test_guest_should_see_search_button_on_the_main_page(self, browser):
        browser.get(link)
        browser.find_element_by_css_selector("input.btn.btn-default")

# Запустите тесты. Здесь мы добавили символ X в параметр -r, чтобы получить подробную информацию по XPASS-тестам:
# pytest -rX -v test_fixture10b_355.py
# И изучите отчёт

In [ ]:
# Дополнительно об использовании этих меток можно почитать в документации:
# Skip and xfail: dealing with tests that cannot succeed.
#     Там есть много разных интересных особенностей, например, как пропускать тест только при выполнении условия,
#     как сделать так, чтобы внезапно прошедший xfailed тест в отчете стал красным, и так далее. 

In [ ]:
# 3_5_6 Задание: пропуск тестов
# Изучите самостоятельно документацию про маркировку xfail.
# Найдите там параметр, который в случае неожиданного прохождения теста, помеченного как xfail,
# отметит в отчете этот тест как упавший. Пометьте таким образом первый тест из этого тестового набора.

# test_xfail_356.py:

import pytest

@pytest.mark.xfail(strict=True)
def test_succeed():
    assert True

@pytest.mark.xfail
def test_not_succeed():
    assert False

@pytest.mark.skip
def test_skipped():
    assert False
    
# Запустите полученные тесты. Обратите внимание на статус прогона тестов.
# Найдите последнюю строчку с итогами запуска, скопируйте текст между символами ===
# и отправьте его в качестве ответа на это задание. 

In [ ]:
# 3_5_6 Задание: пропуск тестов
# сondition - (условие) - условие при котором тестовая функция помечается как xfail (не работающая),
#     принимает либо булево значение либо строку, при булевом значении,
#     нужно будет указать дополнительный параметр reason(причина)
# reason - (причина) - причина по которой фунция помечена как xfail, принимается строка
# raises - (выбрасывания исключений) подкласс исключений которые ожидаются при вызове теста, другие исключения завалят тест
# run - (запуск) - параметр отвечающий за запуск теста, при значении False будет всегда xfail и не будет запускаться.
#     принимается булевы значения
# strict - (строгость) значение False по умолчанию,  в выводе терминала  будет xfailed,
#     если функция провалила тест и xpass, если прошла. при значении True в выводе терминала будет xfailed,
#     если функция не прошла тест, но если фунцкция неожиданно прошла тест то будет fail

In [ ]:
# 3_5_6 Задание: пропуск тестов
# Both XFAIL and XPASS don’t fail the test suite, unless the strict keyword-only parameter is passed as True:
# И XFAIL, и XPASS не проваливают набор тестов, если параметр strict, состоящий только из ключевых слов, не передается как True:



@pytest.mark.xfail(strict=True)
def test_function():
    ...
# This will make XPASS (“unexpectedly passing”) results from this test to fail the test suite.
# Это приведет к тому, что результаты XPASS («неожиданно пройден») этого теста не пройдут набор тестов.

In [ ]:
# 3_5_6 Задание: пропуск тестов


In [ ]:
# 3_5_6 Задание: пропуск тестов


In [ ]:
# 3_5_7 Задание: запуск тестов
# В этом задании нам нужно разобраться в хитросплетениях маркировок.
# Мы имеем файл с тестами, которые уже размечены маркерами для разных ситуаций запуска.

# test_task_run_1_357.py

import pytest


class TestMainPage():
    # номер 1
    @pytest.mark.xfail
    @pytest.mark.smoke
    def test_guest_can_login(self, browser):
        assert True

    # номер 2
    @pytest.mark.regression
    def test_guest_can_add_book_from_catalog_to_basket(self, browser):
        assert True


class TestBasket():
    # номер 3
    @pytest.mark.skip(reason="not implemented yet")
    @pytest.mark.smoke
    def test_guest_can_go_to_payment_page(self, browser):
        assert True

    # номер 4
    @pytest.mark.smoke
    def test_guest_can_see_total_price(self, browser):
        assert True


@pytest.mark.skip
class TestBookPage():
    # номер 5
    @pytest.mark.smoke
    def test_guest_can_add_book_to_basket(self, browser):
        assert True

    # номер 6
    @pytest.mark.regression
    def test_guest_can_see_book_price(self, browser):
        assert True


# номер 7
@pytest.mark.beta_users
@pytest.mark.smoke
def test_guest_can_open_gadget_catalogue(browser):
    assert True
    
# Отметьте ниже только те тестовые методы, которые будут найдены и выполнены PyTest при запуске следующей команды: 
# pytest -v -m "smoke and not beta_users" test_task_run_1_357.py

In [ ]:
# Если что:
# Номер 1 - потому что xfail + smoke
# Номер 2 - нет smoke
# Номер 3 - тест скипается (не выполняется)
# Номер 4 - одиночный smoke (подходит под условие)
# Номер 5 - Класс скипается 
# Номер 6 - аналогично 5
# Номер 7 - не подходит под критерии